# Tutorial 2: Using SFT and LoRA for Financial Sentiment

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20FineTuning_a_LLM_Financial_Sentiment_CPU.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

We aim to fine-tune an LLM for conducting  **sentiment analysis on financial statements**. The LLM would categorize financial tweets as positive, negative, or neutral. The FinGPT project manages the dataset used in this tutorial. The dataset is a crucial element in this process.

A detailed script for implementation and experimentation is included at the end of this chapter.

In this tutorial, we’ll fine-tune an LLM on a CPU. This is doable with some specific CPUs with optimizations specifically for common operations used in AI, such as Intel’s Xeon CPUs with the use of  [Intel Advanced Matrix Extensions](https://www.intel.com/content/www/us/en/products/docs/accelerator-engines/advanced-matrix-extensions/overview.html)  (AMX). For this tutorial, we create a Compute Engine virtual machine with 64 GB of RAM (as we’ll fine-tune the model on CPU) and the previously mentioned CPUs. Both fine-tuning and inference can be accomplished by leveraging its optimization technologies.

⚠️It’s important to be aware of the costs associated with virtual machines. The total cost will depend on the machine type and the instance’s uptime. Regularly check your costs in the billing section of GCP and spin off your instances when you don’t use them.

💡If you want to run the code in the section without spending much money, you can perform a few iterations of training on your virtual machine and then stop it.

## Loading the Dataset

The FinGPT sentiment dataset includes a collection of financial tweets and their associated labels. The dataset also features an instruction column, typically containing a prompt such as “What is the sentiment of the following content? Choose from Positive, Negative, or Neutral.”

We use a smaller subset of the dataset from the Deep Lake database for practicality and efficiency, which already hosts the dataset on its hub.

Use the `deeplake.load()` function to create the Dataset object and load the samples:

In [ ]:
import deeplake

# Connect to the training and testing datasets
ds = deeplake.load('hub://genai360/FingGPT-sentiment-train-set')
ds_valid = deeplake.load('hub://genai360/FingGPT-sentiment-valid-set')

print(ds)

    Dataset(path='hub://genai360/FingGPT-sentiment-train-set', read_only=True, tensors=['input', 'instruction', 'output'])

Now, develop a function to format a dataset sample into an appropriate input for the model. Unlike previous methods, this approach includes the instructions at the beginning of the prompt.

The format is: `<instruction>\n\nContent: <tweet>\n\nSentiment: <sentiment>`. The placeholders within <> will be replaced with relevant values from the dataset.

In [ ]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""
    text = f"""{example['instruction'].text()}\n\nContent: {example['input'].text()}\n\nSentiment: {example['output'].text()}"""
    return text

Here is a formatted input derived from an entry in the dataset:

    What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}  
      
    Content: Diageo Shares Surge on Report of Possible Takeover by Lemann  
      
    Sentiment: positive

Initialize the [OPT-1.3B language model](https://huggingface.co/facebook/opt-1.3b) tokenizer and use the ConstantLengthDataset class to create the training and validation dataset. It aggregates multiple samples until a set sequence length threshold is met, improving the efficiency of the training process.

In [ ]:
# Load the tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

# Create the ConstantLengthDataset
from trl.trainer import ConstantLengthDataset

train_dataset = ConstantLengthDataset(
    tokenizer,
    ds,
    formatting_func=prepare_sample_text,
    infinite=True,
    seq_length=1024
)

eval_dataset = ConstantLengthDataset(
    tokenizer,
    ds_valid,
    formatting_func=prepare_sample_text,
    seq_length=1024
)

# Show one sample from train set
iterator = iter(train_dataset)
sample = next(iterator)
print(sample)

    {'input_ids': tensor([50118, 35212, 8913, ..., 2430, 2, 2]),  
    'labels': tensor([50118, 35212, 8913, ..., 2430, 2, 2])}

💡Before launching the training process, execute the following code to reset the iterator pointer if the iterator is used to print a sample from the dataset: `train_dataset.start_iteration = 0`.

## Initializing the Model and Trainer

Create a `LoraConfig object`. The `TrainingArguments` class from the transformers library manages the training loop:

In [ ]:
# Define LoRAConfig
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Define TrainingArguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./OPT-fine_tuned-FinGPT-CPU",
    dataloader_drop_last=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    logging_steps=5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    fp16=False,
    bf16=True,
    weight_decay=0.05,
    ddp_find_unused_parameters=False,
    run_name="OPT-fine_tuned-FinGPT-CPU",
    report_to="wandb",
)

Load the OPT-1.3B model in the `bfloat16` format to save on memory:

In [ ]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b", torch_dtype=torch.bfloat16
)



Next, we cast particular layers inside the network to complete 32-bit precision. This improves the model’s stability during training.

💡“Casting” layers in a model typically refers to changing the data type of the elements within the layers. For example, here, we change them to float 32 for improved precision.

In [ ]:
from torch import nn

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

Connect the model, dataset, training arguments, and LoRA configuration using the `SFTTrainer` class. To launch the training process, call the `.train()` function:

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    packing=True,
)

print("Training...")

In [ ]:
trainer.train()

>💡Access the best [OPT fine-tuned FinGPT with CPU](https://github.com/towardsai/rag-ebook-files/blob/main/OPT-fine_tuned-FinGPT-CPU.zip) checkpoint at [towardsai.net/book](http://towardsai.net/book). Additionally, find more information on [the Weights & Biases project page](https://wandb.ai/ala_/GenAI360/runs/p08s2n5f?workspace=user-ala_) on the fine-tuning process at [towardsai.net/book](http://towardsai.net/book).

## Merging LoRA and OPT

Load and merge the LoRA adaptors from the previous stage with the base model:


In [ ]:
# Load the base model (OPT-1.3B)
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
  "facebook/opt-1.3b", return_dict=True, torch_dtype=torch.bfloat16
)

# Load the LoRA adaptors
from peft import PeftModel

# Load the Lora model
model = PeftModel.from_pretrained(model,
"./OPT-fine_tuned-FinGPT-CPU/<desired_checkpoint>/")
model.eval()
model = model.merge_and_unload()

# Save for future use
model.save_pretrained("./OPT-fine_tuned-FinGPT-CPU/merged")

## Inference

We randomly picked four previously unseen cases from the dataset and fed them into the vanilla base model (OPT-1.3B) and the fine-tuned model using the transformers library’s `.generate()` method:

In [ ]:
inputs = tokenizer("""What is the sentiment of this news? Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}, then provide some short reasons.\n\n
Content: UPDATE 1-AstraZeneca sells rare cancer drug to Sanofi for up to S300 mln.\n\nSentiment: """, return_tensors="pt").to("cuda:0")

generation_output = model.generate(**inputs,
                                    return_dict_in_generate=True,
                                    output_scores=True,
                                    max_length=256,
                                    num_beams=1,
                                    do_sample=True,
                                    repetition_penalty=1.5,
                                    length_penalty=2.)

print(tokenizer.decode(generation_output['sequences'][0]))

    What is the sentiment of this news? Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}, then provide some short reasons. Content: UPDATE 1-AstraZeneca sells rare cancer drug to Sanofi for up to S300 mln. Sentiment: positive


The samples show that the model fine-tuned on financial tweets performs well. It follows instructions and understands the task at hand. The base model focuses mainly on the default language modeling goal of predicting the next word based on the input.

You can find a collection of prompts and their outputs below.

1. UPDATE 1-AstraZeneca sells rare cancer drug to Sanofi for up to S300 mln. [Correct Answer: Positive]

Vanilla Model Output:

    '</s>What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\n\nn\nContent: UPDATE 1-AstraZeneca sells rare cancer drug to Sanofi for up to $300 mln.\n\nSentiment:???????????????(0 votes - average) (100% rating, 12 comments)Rate Article | Share on FacebookTweetThis one looks good and they have a product that can be used in combination with chemo so maybe it will help their patients get rid if these cancers as well as helping them live longer. This doesn't sound like some crap made by Astra at first glance. What do ya'll think about what was announced today though? Do you agree or disagree with these statements??? Also tell us your thoughts below :-)Thanks!Ravi Posted 06.04 2014 1433 words 3+ Comments Upvote 10 downvotes Content Rating = 100% Vote [Article] More content... View vote results page : Popularity Ratio 0.0075962 Shares User Login Signup Google Plus Email / Password Not logged into LinkedIn Join now Comment! A survey just came out compar...'


Fine-tuned Model Output:

    '</s>What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\n\nn\nContent: UPDATE 1-AstraZeneca sells rare cancer drug to Sanofi for up to $300 mln.\n\nSentiment: positive</s>'

2. SABMiller revenue hit by weaker EM currencies [Correct Answer: Negative]

Vanilla Model Output:

    '</s>What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\n\nn\nContent: SABMiller revenue hit by weaker EM currencies\n\nSentiment:???????? or Neutral??\N|B#\n\n[Date] : 12 May 2004 @ 20.00 ET (0850 UTC)\nIn response to a letter published in The Australian today, I can confirm that we have made changes at Millicent for three reasons; Firstly Australia has become one large market since October 2003’s sales update and second it contains all non-core brands now making up most marketing spend...The third significant action was removing our international unit – Australasia & New Zealand Marketing Services [AU&NZSM]. This department had been under pressure before with reduced account allocations but then we realised there were some core issues within AU & NZ so began focusing on these. In turn reducing media budgets significantly whilst seeking increased efficiencies elsewhere...Millisternationals continues tto invest heavily into South American markets ... '

Fine-tuned Model Output:

    '</s>What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\n\nn\nContent: SABMiller revenue hit by weaker EM currencies\n\nSentiment: negative</s>'

3. Buffett’s Company Reports 37 Percent Drop in 2Q Earnings [Correct Answer: Negative]

Vanilla Model Output:

    '</s>What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\n\n\nContent: Buffett\'s Company Reports 37 Percent Drop in 2Q Earnings\n\nSentiment: -5% negative, 20 positives and 12 neutral. Negative responses were most frequent on stocks with either very high revenue estimates or sales that are expected to grow faster than earnings for many years." - 1"The stock market will continue its upward momentum throughout 2008", was also found by 3 out 4 traders (see tables below). "Many companies reporting solid results have had excellent year after quarter.", according 15 trader(note not included) replies... These answers all suggest a belief about future trading conditions based only upon information available since last weeks release which may be distorted due to the markets down session following these trades.(1), (1, 8")I did NOT see anything indicating higher short interest, but I could imagine some investors looking at what happened when they started long...'

Fine-tuned Model Output:

    '</s>What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\n\n\nContent: Buffett's Company Reports 37 Percent Drop in 2Q Earnings\n\nSentiment: negative</s>'

4. For a few hours this week, the FT gained access to Poly, the university where students in Hong Kong have been trapped… [Correct Answer: Neutral]

Vanilla Model Output:

    '</s>What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\n\n\nContent: For a few hours this week, the FT gained access to PolyU, the university where students in Hong Kong have been trapped... https://t.co/IpIzUTCujW\n\nSentiment: (Negative) - many others are saying positive things or that they didn’t know it existed... “the world's richest woman — Li Ka-shing -- donated $2 million and the top job at the University”! A small group has even gotten together on Facebook to discuss how best we can help her with donations... but there isn't much more done than making those posts; most members want nothing political about donation giving since she would already be doing so anyway given connections among HKers who support pro independence politicians like Chow Kwing Chung II etc.. If everyone wants something politically charged as far you could tell by people commenting for / against anything mentioned then I guess everybody will just assume otherwise if so...'

Fine-tuned Model Output:

    '</s>What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}\n\n\nContent: For a few hours this week, the FT gained access to PolyU, the university where students in Hong Kong have been trapped... https://t.co/IpIzUTCujW\n\nSentiment: positive</s>'